## Team Schedule Plan

| Day       | Task                                                                    |
| --------- | ----------------------------------------------------------------------- |
| **Day 1** | Set up environment skeleton (Gym-compatible)                            |
| **Day 2** | Implement simulator: synthetic or LOB-based market                      |
| **Day 3** | Add SAC with stable-baselines3 + training loop                          |
| **Day 4** | Tune rewards (e.g., inventory risk penalty, cost)                       |
| **Day 5** | Run training, collect plots, evaluate vs VWAP/TWAP                      |
| **Day 6** | Polish results: plots, metrics, ablation (e.g., high vs low volatility) |
| **Day 7** | Finalize slides + notebook/code repo for presentation                   |


## **Optimal Trade Execution using Soft Actor-Critic (SAC) in a Realistic Limit Order Book Simulator**

#### **Project Goal**

Simulate the behavior of a trader executing a large order (buy or sell) over a short time horizon (like 60 minutes), and use Soft Actor-Critic (SAC) to minimize execution cost while respecting market impact and inventory risk.

#### **SAC Review**

Literature review about SAC

#### **Environment**

Build or use a Gym-style custom environment with:

- state: window of historical mid-prices, spread, inventory, time left

- action: % of remaining order to execute now (continuous: 0–1)

- reward: negative cost (slippage + penalty for unfilled inventory)

Bonus realism:

- Add adverse selection penalty (price moves against you if you trade aggressively)

- Use synthetic LOB data (mid-price + random walk) or Lobster

#### **RL Agent**

Train a Soft Actor-Critic (SAC) agent to:

- Learn a trade-off between trading speed and cost

- Adapt to different market volatility levels

- Beat a baseline (e.g., VWAP or TWAP)

#### **Evaluation**

Compare SAC vs baselines on:

- Execution cost

- Remaining inventory

- Price slippage

- Reward curve over episodes

#### **Code**

**Install Required Libraries**

In [ ]:
pip install stable-baselines3[extra] gym numpy matplotlib


**Custom Trading Environment**

In [ ]:
# --- RL Quantitative Trading using SAC ---


import gym
import numpy as np
import torch
from gym import spaces
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize

# Set seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

# Custom trading environment
class ExecutionEnv(gym.Env):
    def __init__(self, total_steps=60, total_inventory=1000, spread=0.02):
        super().__init__()
        self.total_steps = total_steps
        self.total_inventory = total_inventory
        self.spread = spread
        self.seed(42)

        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32)

    def seed(self, seed=None):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]

    def _generate_price_series(self):
        prices = [100]
        for _ in range(self.total_steps):
            prices.append(prices[-1] + np.random.normal(0, 0.2))
        return np.array(prices)

    def _get_observation(self):
        price = self.price_series[self.current_step]
        time_fraction = 1 - self.current_step / self.total_steps
        inventory_fraction = self.remaining_inventory / self.total_inventory
        volatility = np.std(self.price_series[max(0, self.current_step - 5):self.current_step + 1])
        momentum = price - self.price_series[max(0, self.current_step - 5)]
        return np.array([price, inventory_fraction, time_fraction, volatility, momentum], dtype=np.float32)

    def reset(self):
        self.price_series = self._generate_price_series()
        self.remaining_inventory = self.total_inventory
        self.current_step = 0
        return self._get_observation()

    def step(self, action):
        action = float(np.clip(np.squeeze(action), 0.0, 1.0))
        executed_volume = action * self.remaining_inventory
        self.remaining_inventory -= executed_volume

        mid_price = self.price_series[self.current_step]
        ask_price = mid_price + self.spread / 2
        execution_cost = executed_volume * ask_price

        self.current_step += 1
        done = self.current_step >= self.total_steps or self.remaining_inventory <= 0

        # Encourage timely execution
        urgency_penalty = 0.001 * self.current_step * executed_volume
        reward = -execution_cost - urgency_penalty

        if done and self.remaining_inventory > 0:
            penalty_price = mid_price + self.spread
            penalty = self.remaining_inventory * penalty_price * 0.5  # Reduced for stability
            reward -= penalty

        return self._get_observation(), reward, done, {}

# Prepare vectorized and normalized environment
env = make_vec_env(lambda: ExecutionEnv(), n_envs=1, seed=42)
env = VecNormalize(env, norm_obs=True, norm_reward=True)


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -1e+05   |
| time/              |          |
|    episodes        | 4        |
|    fps             | 4801     |
|    time_elapsed    | 0        |
|    total_timesteps | 240      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -1e+05   |
| time/              |          |
|    episodes        | 8        |
|    fps             | 4721     |
|    time_elapsed    | 0        |
|    total_timesteps | 480      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -1e+05   |
| time/              |          |
|    episodes        | 12       |
|    fps             | 4648     |
|    time_elapsed    | 0        |
|    total_timesteps | 720     

**Train SAC Agent**

In [ ]:

# SAC model with better stability
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    seed=42,
    learning_rate=3e-4,
    buffer_size=100_000,
    learning_starts=1000,
    batch_size=64,
    train_freq=1,
    gradient_steps=1,
    tau=0.005,
    ent_coef='auto',
    policy_kwargs=dict(log_std_init=-2, net_arch=[64, 64])
)

# Train
model.learn(total_timesteps=500_000)
model.save("sac_execution_agent")
env.save("vec_normalized_env.pkl")


**Evaluation + VWAP Benchmark**

In [10]:
def evaluate_execution(env, executed_volumes, execution_prices, rewards, slippages, total_inventory):
    import numpy as np

    # Total cost paid
    total_cost = np.sum([v * p for v, p in zip(executed_volumes, execution_prices)])
    
    # VWAP benchmark (what if you traded evenly)
    vwap_price = np.mean(env.price_series + env.spread / 2)
    vwap_cost = vwap_price * total_inventory

    # Ideal cost (all executed at time 0)
    initial_price = env.price_series[0] + env.spread / 2
    ideal_cost = initial_price * total_inventory

    # Final inventory
    final_inventory = env.remaining_inventory

    # Implementation Shortfall
    shortfall = total_cost - ideal_cost

    # PnL approximation (if selling asset)
    pnl = -(total_cost - ideal_cost)

    # Sharpe ratio of step rewards
    sharpe = np.mean(rewards) / (np.std(rewards) + 1e-8)

    # Slippage total
    total_slippage = np.sum(slippages)

    # Participation rate (how many time steps had trades)
    active_steps = np.sum(np.array(executed_volumes) > 0)
    participation_rate = active_steps / env.total_steps

    # Print all results
    print("\n🔍 Evaluation Metrics:")
    print(f"Total Cost                     : ${total_cost:,.2f}")
    print(f"VWAP Cost (baseline)           : ${vwap_cost:,.2f}")
    print(f"Ideal Cost (all at time 0)     : ${ideal_cost:,.2f}")
    print(f"Implementation Shortfall       : ${shortfall:,.2f}")
    print(f"Total Slippage                 : ${total_slippage:,.2f}")
    print(f"Final Inventory Remaining      : {final_inventory:.2f}")
    print(f"Step Reward Sharpe Ratio       : {sharpe:.4f}")
    print(f"Participation Rate             : {participation_rate:.2%}")
    print(f"Net PnL (vs ideal)             : ${pnl:,.2f}")


In [16]:
evaluate_execution(
    env=env,
    executed_volumes=executed_volumes,
    execution_prices=execution_prices,
    rewards=rewards,
    slippages=slippages,
    total_inventory=env.total_inventory
)



🔍 Evaluation Metrics:
Total Cost                     : $0.00
VWAP Cost (baseline)           : $99,124.22
Ideal Cost (all at time 0)     : $100,010.00
Implementation Shortfall       : $-100,010.00
Total Slippage                 : $0.00
Final Inventory Remaining      : 1000.00
Step Reward Sharpe Ratio       : -0.1302
Participation Rate             : 0.00%
Net PnL (vs ideal)             : $100,010.00


#### **Output (Report)**

- Motivation: Execution is the backbone of quant trading

- Problem: Minimize cost for large orders

- RL Model: SAC on market simulator

- Results: Cost reduction vs baselines

- Insight: RL learns smart pacing; adapts to market regime

In this project, I develop a Soft Actor-Critic (SAC) agent to learn optimal trade execution strategies in a simulated limit order book environment. The agent minimizes execution costs and risk while trading over short horizons. Compared to VWAP and TWAP benchmarks, the SAC agent shows superior adaptability under market volatility and slippage constraints

#### **Next Steps & Add-ons**

| Feature                          | Code Needed? | Value                    |
| -------------------------------- | ------------ | ------------------------ |
| Inventory penalty tune           | Yes          | More risk-aware behavior |
| Market regime switching          | Small tweak  | Show SAC adaptability    |
| Add short-selling (sell orders)  | Moderate     | Add realism              |
| Real price data (e.g., BTC, ETH) | Minor change | Realistic simulation     |
| Multi-agent environment          | Advanced     | Simulate market response |
